In [29]:
import json
import pandas
import os
import re
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from  itertools import tee, chain, islice
from math import log
from operator import itemgetter
from itertools import takewhile
import requests
from operator import itemgetter 

In [30]:
def window(it, size=2):
    yield from zip(*[islice(it, s, None) for s, it in enumerate(tee(it, size))])

In [31]:
def read_tagging_line(line):
    _id, tag_line = line.split(';')
    tags = tag_line.split(' ')
    non_empty_tags =  filter(lambda s: s,tags)
    return _id, list(non_empty_tags)
    
def read_tagging():
   return map(read_tagging_line, open('tagging-2018','r'))

def get_class(unigram):
    return unigram.split(':')[1]

def filter_bigram_by_class(bigram):
    return get_class(bigram[0]) in ['subst'] and get_class(bigram[1]) in ['subst','adj']

In [32]:
lines = map( itemgetter(1), read_tagging())
unigrams_it1, unigrams_it2 = tee(lines)

In [33]:
unigrams =  chain.from_iterable(unigrams_it1)
bigrams = chain.from_iterable( map( lambda it: window(it,2),unigrams_it2))

In [34]:
counted_unigrams= Counter(unigrams)
counted_bigrams= Counter(bigrams)

unigram_count = sum(counted_unigrams.values())
bigram_count =  sum(counted_bigrams.values())

# 10 najpopularniejszych bigramów w 2018

In [35]:
counted_bigrams.most_common(10)

[(('z:prep', 'dzień:subst'), 24668),
 (('artykuł:brev', 'koło:brev'), 14250),
 (('w:prep', 'dzień:subst'), 10874),
 (('w:prep', 'sprawa:subst'), 10384),
 (('na:prep', 'podstawa:subst'), 9424),
 (('z:prep', 'artykuł:brev'), 8892),
 (('koło:brev', 'pan:brev'), 8854),
 (('w:prep', 'ten:adj'), 8302),
 (('kwota:subst', 'złoty:brev'), 7886),
 (('ubezpieczenie:subst', 'społeczny:adj'), 7780)]

# 10 najpopularniejszych unigramów w 2018

In [36]:
counted_unigrams.most_common(10) 

[('w:prep', 229406),
 ('z:prep', 143952),
 ('na:prep', 85786),
 ('i:conj', 78166),
 ('do:prep', 75758),
 ('nie:qub', 59842),
 ('dzień:subst', 52958),
 ('on:ppron3', 51864),
 ('o:prep', 49760),
 ('rok:brev', 49266)]

In [37]:
from  python_llr.llr import llr_2x2
def loglikelihood(bigram):
   k11 = counted_bigrams[bigram] 
   k12 = counted_unigrams[bigram[0]] - counted_bigrams[bigram]
   k21 = counted_unigrams[bigram[1]] - counted_bigrams[bigram]
   k22 = bigram_count - (counted_unigrams[bigram[0]] + counted_unigrams[bigram[1]] - counted_bigrams[bigram])
   return llr_2x2(k11,k12,k21,k22) 

# top 30 bigrams by loglikelihood

In [38]:
filtered =list(filter(filter_bigram_by_class, counted_bigrams.keys()))
log_like_vals =map(lambda bigram: (bigram, loglikelihood(bigram)) , filtered)
list(sorted(log_like_vals,key = itemgetter(1),reverse=True))[:30]

[(('ubezpieczenie:subst', 'społeczny:adj'), 90171.24090331813),
 (('sąd:subst', 'rejonowy:adj'), 66100.29170284118),
 (('sąd:subst', 'okręgowy:adj'), 64060.64552685816),
 (('organ:subst', 'rentowy:adj'), 51036.95560479551),
 (('zeznanie:subst', 'świadek:subst'), 41783.675160848186),
 (('materiał:subst', 'dowodowy:adj'), 38865.34503389546),
 (('dzień:subst', 'styczeń:subst'), 35115.35953170154),
 (('sąd:subst', 'wysoki:adj'), 33954.72247091122),
 (('skarb:subst', 'państwo:subst'), 31911.598054927555),
 (('dzień:subst', 'grudzień:subst'), 26934.586194734555),
 (('dzień:subst', 'październik:subst'), 26426.55840735021),
 (('stan:subst', 'faktyczny:adj'), 26180.792122840125),
 (('płyta:subst', 'cd:subst'), 24264.958803717625),
 (('koszt:subst', 'proces:subst'), 22435.99068148405),
 (('działalność:subst', 'gospodarczy:adj'), 21883.471563419924),
 (('płatnik:subst', 'składka:subst'), 21649.62450915556),
 (('dzień:subst', 'listopad:subst'), 21634.893700654153),
 (('dzień:subst', 'lipiec:subst'